In [56]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#1. 데이터 읽기

In [57]:
moviefile = '/content/drive/MyDrive/Colab Notebooks/Lec4_BigData/6.협업필터링/data/data/movies.csv'
rating = '/content/drive/MyDrive/Colab Notebooks/Lec4_BigData/6.협업필터링/data/data/ratings.csv'

df_movies = pd.read_csv(moviefile)
df_ratings= pd.read_csv(rating)
df_ratings.shape

(100836, 4)

#2. 데이터클린징

In [58]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [59]:
df_ratings.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


> 이상치 및 결측치는 없는것으로 나타난다.

In [60]:
from datetime import datetime
for idx in range(5):
  print(datetime.fromtimestamp(df_ratings['timestamp'][idx]))

2000-07-30 18:45:03
2000-07-30 18:20:47
2000-07-30 18:37:04
2000-07-30 19:03:35
2000-07-30 18:48:51


In [61]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [62]:
df_ratings_movies = pd.merge(df_ratings, df_movies, on='movieId')
df_ratings_movies_pivot = df_ratings_movies.pivot_table('rating', index='userId', columns='title')
df_ratings_movies_pivot = df_ratings_movies_pivot.fillna(0)
df_ratings_movies_pivot

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...All the Marbles (1981),...And Justice for All (1979),00 Schneider - Jagd auf Nihil Baxter (1994),1-900 (06) (1994),10 (1979),10 Cent Pistol (2015),10 Cloverfield Lane (2016),10 Items or Less (2006),10 Things I Hate About You (1999),10 Years (2011),"10,000 BC (2008)",100 Girls (2000),100 Streets (2016),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),101 Dalmatians II: Patch's London Adventure (2003),101 Reykjavik (101 Reykjavík) (2000),102 Dalmatians (2000),10th & Wolf (2006),"10th Kingdom, The (2000)","10th Victim, The (La decima vittima) (1965)","11'09""01 - September 11 (2002)",11:14 (2003),"11th Hour, The (2007)",12 Angry Men (1957),12 Angry Men (1997),12 Chairs (1971),12 Chairs (1976),12 Rounds (2009),12 Years a Slave (2013),...,Zathura (2005),Zatoichi and the Chest of Gold (Zatôichi senryô-kubi) (Zatôichi 6) (1964),Zazie dans le métro (1960),Zebraman (2004),"Zed & Two Noughts, A (1985)",Zeitgeist: Addendum (2008),Zeitgeist: Moving Forward (2011),Zeitgeist: The Movie (2007),Zelary (2003),Zelig (1983),Zero Dark Thirty (2012),Zero Effect (1998),"Zero Theorem, The (2013)",Zero de conduite (Zero for Conduct) (Zéro de conduite: Jeunes diables au collège) (1933),Zeus and Roxanne (1997),Zipper (2015),Zodiac (2007),Zombeavers (2014),Zombie (a.k.a. Zombie 2: The Dead Are Among Us) (Zombi 2) (1979),Zombie Strippers! (2008),Zombieland (2009),Zone 39 (1997),"Zone, The (La Zona) (2007)",Zookeeper (2011),Zoolander (2001),Zoolander 2 (2016),Zoom (2006),Zoom (2015),Zootopia (2016),Zulu (1964),Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

#3. 영화 간 유사도 계산하기

## 3.1 사용자 기반 추천
> 위의 DataFrame을 이용하여 사용자 1번과 성향이 가장 비슷한 순으로 10명을 출력하시오.

In [63]:
def getSimUser(data_frame, user_name, topn = 10):
  from sklearn.metrics.pairwise import cosine_similarity

  user_similarity = cosine_similarity(data_frame, data_frame)
  df_user_sim = pd.DataFrame(user_similarity, columns=data_frame.index, index=data_frame.index)
  sim_topn = df_user_sim[user_name].sort_values(ascending=False)[1:topn+1]
  return  sim_topn


getSimUser(df_ratings_movies_pivot, 1)

userId
266    0.357408
313    0.351562
368    0.345127
57     0.345034
91     0.334727
469    0.330664
39     0.329782
288    0.329700
452    0.328048
45     0.327922
Name: 1, dtype: float64

In [64]:
df_ratings_movies_pivot.loc[266].sort_values(ascending=False)[:10]

title
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)    5.0
Crow, The (1994)                                                                  5.0
Goodfellas (1990)                                                                 5.0
Untouchables, The (1987)                                                          5.0
Dirty Work (1998)                                                                 5.0
Fear and Loathing in Las Vegas (1998)                                             5.0
Ref, The (1994)                                                                   5.0
Matrix, The (1999)                                                                5.0
This Is Spinal Tap (1984)                                                         5.0
Time Bandits (1981)                                                               5.0
Name: 266, dtype: float64

> 10개의 영화만 봐서는 둘의 성향이 비슷하다는 것을 알 수는 없다.

In [65]:
print(type(df_ratings_movies_pivot.loc[1]))
df_ratings_movies_pivot.loc[1].index

<class 'pandas.core.series.Series'>


Index([''71 (2014)', ''Hellboy': The Seeds of Creation (2004)',
       ''Round Midnight (1986)', ''Salem's Lot (2004)',
       ''Til There Was You (1997)', ''Tis the Season for Love (2015)',
       ''burbs, The (1989)', ''night Mother (1986)',
       '(500) Days of Summer (2009)', '*batteries not included (1987)',
       ...
       'Zulu (2013)', '[REC] (2007)', '[REC]² (2009)',
       '[REC]³ 3 Génesis (2012)',
       'anohana: The Flower We Saw That Day - The Movie (2013)',
       'eXistenZ (1999)', 'xXx (2002)', 'xXx: State of the Union (2005)',
       '¡Three Amigos! (1986)', 'À nous la liberté (Freedom for Us) (1931)'],
      dtype='object', name='title', length=9719)

> 위의 코드를 DataFrame으로 변환하여 출력하시오

In [66]:
pd.DataFrame(df_ratings_movies_pivot.loc[1].values, index=df_ratings_movies_pivot.loc[1].index).transpose()


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...All the Marbles (1981),...And Justice for All (1979),00 Schneider - Jagd auf Nihil Baxter (1994),1-900 (06) (1994),10 (1979),10 Cent Pistol (2015),10 Cloverfield Lane (2016),10 Items or Less (2006),10 Things I Hate About You (1999),10 Years (2011),"10,000 BC (2008)",100 Girls (2000),100 Streets (2016),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),101 Dalmatians II: Patch's London Adventure (2003),101 Reykjavik (101 Reykjavík) (2000),102 Dalmatians (2000),10th & Wolf (2006),"10th Kingdom, The (2000)","10th Victim, The (La decima vittima) (1965)","11'09""01 - September 11 (2002)",11:14 (2003),"11th Hour, The (2007)",12 Angry Men (1957),12 Angry Men (1997),12 Chairs (1971),12 Chairs (1976),12 Rounds (2009),12 Years a Slave (2013),...,Zathura (2005),Zatoichi and the Chest of Gold (Zatôichi senryô-kubi) (Zatôichi 6) (1964),Zazie dans le métro (1960),Zebraman (2004),"Zed & Two Noughts, A (1985)",Zeitgeist: Addendum (2008),Zeitgeist: Moving Forward (2011),Zeitgeist: The Movie (2007),Zelary (2003),Zelig (1983),Zero Dark Thirty (2012),Zero Effect (1998),"Zero Theorem, The (2013)",Zero de conduite (Zero for Conduct) (Zéro de conduite: Jeunes diables au collège) (1933),Zeus and Roxanne (1997),Zipper (2015),Zodiac (2007),Zombeavers (2014),Zombie (a.k.a. Zombie 2: The Dead Are Among Us) (Zombi 2) (1979),Zombie Strippers! (2008),Zombieland (2009),Zone 39 (1997),"Zone, The (La Zona) (2007)",Zookeeper (2011),Zoolander (2001),Zoolander 2 (2016),Zoom (2006),Zoom (2015),Zootopia (2016),Zulu (1964),Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0


In [67]:
def get_score_movie(data_frame, userid):
  #userId에 따른 영화 추천
  user_movie = data_frame.loc[userid]
  #dataframe으로 변환
  movie_list = [user_movie.index, user_movie.values]
  df_movies_score = pd.DataFrame(data=movie_list)
  df_movies_score.index = ['title', 'score']
  #return df_movies_score.transpose()

  #series를 DataFrame으로 변환
  df_movies_score = pd.DataFrame(data={'title':user_movie.index, 'score':user_movie.values})
  return df_movies_score.reset_index()
  
  
get_score_movie(df_ratings_movies_pivot, 1)

,index,title,score
0,0,'71 (2014),0.0
1,1,'Hellboy': The Seeds of Creation (2004),0.0
2,2,'Round Midnight (1986),0.0
3,3,'Salem's Lot (2004),0.0
4,4,'Til There Was You (1997),0.0
...,...,...,...
9714,9714,eXistenZ (1999),0.0
9715,9715,xXx (2002),0.0
9716,9716,xXx: State of the Union (2005),0.0
9717,9717,¡Three Amigos! (1986),4.0


In [68]:
df_user1_movies = get_score_movie(df_ratings_movies_pivot, 1)

# user266의 모든 영화 목록
df_user266_movies = get_score_movie(df_ratings_movies_pivot, 266)
df_user266_movies

,index,title,score
0,0,'71 (2014),0.0
1,1,'Hellboy': The Seeds of Creation (2004),0.0
2,2,'Round Midnight (1986),0.0
3,3,'Salem's Lot (2004),0.0
4,4,'Til There Was You (1997),0.0
...,...,...,...
9714,9714,eXistenZ (1999),0.0
9715,9715,xXx (2002),0.0
9716,9716,xXx: State of the Union (2005),0.0
9717,9717,¡Three Amigos! (1986),0.0


###3.1.1 Quiz
> user1이 안본 영화 중 user266이 본 4.5이상의 상위 10개 영화를 추천하세요

In [69]:
user1_play_list = df_user1_movies[df_user1_movies['score']>0]['title'].values

df_user266_top10 = df_user266_movies[(~df_user266_movies['title'].isin(user1_play_list)) & (df_user266_movies['score']>=4.5)].sort_values('score', ascending=False)
df_user266_top10.title.values.tolist()

['2001: A Space Odyssey (1968)',
 'Stripes (1981)',
 'Producers, The (1968)',
 'Raising Arizona (1987)',
 'Real Genius (1985)',
 'Ref, The (1994)',
 'Roger & Me (1989)',
 'Ronin (1998)',
 'Summer of Sam (1999)',
 'Heathers (1989)',
 'This Is Spinal Tap (1984)',
 'Three Kings (1999)',
 'Time Bandits (1981)',
 'Trainspotting (1996)',
 'True Romance (1993)',
 'Unforgiven (1992)',
 'Hunt for Red October, The (1990)',
 'Hard-Boiled (Lat sau san taam) (1992)',
 'Aliens (1986)',
 'Dirty Work (1998)',
 'Austin Powers: The Spy Who Shagged Me (1999)',
 'Blade Runner (1982)',
 'Brazil (1985)',
 'Casino (1995)',
 'Commitments, The (1991)',
 'Crow, The (1994)',
 'Donnie Brasco (1997)',
 'Friday (1995)',
 'Election (1999)',
 'Evil Dead II (Dead by Dawn) (1987)',
 'Eyes Wide Shut (1999)',
 'Fear and Loathing in Las Vegas (1998)',
 'Fish Called Wanda, A (1988)',
 'Fisher King, The (1991)',
 'Untouchables, The (1987)']

##3.2 다수의 사용자 기반 추천
> 1. 유사도가 높은 사용자 추출
2. 추천 받을 사람의 안본 영화 추출
3. 추천하는 사람의 4점 이상 영화 추출
4. 2, 3번을 이용한 추천 영화 목록 만들기
5. 2~4 를 반복하여 모든 사람의 영화목록 정보 저장
6. 빈도수 구하기
7. 빈도수가 높은 영화 추천

###3.2.1 유사도가 높은 사용자 추출

In [70]:
from sklearn.metrics.pairwise import cosine_similarity
TARGET_USER = 1

def get_user_similarity(data_frame, user_name, topn = 10):
  user_similarity = cosine_similarity(data_frame, data_frame)
  df_user_sim = pd.DataFrame(user_similarity, columns=data_frame.index, index=data_frame.index)
  sim_topn = df_user_sim[user_name].sort_values(ascending=False)[1:topn+1]
  return  sim_topn


get_user_similarity(df_ratings_movies_pivot, TARGET_USER)

userId
266    0.357408
313    0.351562
368    0.345127
57     0.345034
91     0.334727
469    0.330664
39     0.329782
288    0.329700
452    0.328048
45     0.327922
Name: 1, dtype: float64

###3.2.2 추천 받을 사람의 안본영화 추출

In [71]:
def get_score_movie(data_frame, userid):
  #userId에 따른 영화 추천
  user_movie = data_frame.loc[userid]

  #series를 DataFrame으로 변환
  df_movies_score = pd.DataFrame(data={'title':user_movie.index, 'score':user_movie.values})
  df_movies_score = df_movies_score.reset_index()

  return df_movies_score[df_movies_score['score']==0]

get_score_movie(df_ratings_movies_pivot, TARGET_USER)

,index,title,score
0,0,'71 (2014),0.0
1,1,'Hellboy': The Seeds of Creation (2004),0.0
2,2,'Round Midnight (1986),0.0
3,3,'Salem's Lot (2004),0.0
4,4,'Til There Was You (1997),0.0
...,...,...,...
9713,9713,anohana: The Flower We Saw That Day - The Movi...,0.0
9714,9714,eXistenZ (1999),0.0
9715,9715,xXx (2002),0.0
9716,9716,xXx: State of the Union (2005),0.0


###3.2.3  추천하는 사람의 4점 이상영화 추출

In [72]:
def get_score_movie(data_frame, userid):
  #userId에 따른 영화 추천
  user_movie = data_frame.loc[userid]

  #series를 DataFrame으로 변환
  df_movies_score = pd.DataFrame(data={'title':user_movie.index, 'score':user_movie.values})
  df_movies_score = df_movies_score.reset_index()

  return df_movies_score[df_movies_score['score']>=4]

get_score_movie(df_ratings_movies_pivot, TARGET_USER)

,index,title,score
48,48,"13th Warrior, The (1999)",4.0
66,66,20 Dates (1998),4.0
202,202,"Abyss, The (1989)",4.0
245,245,"Adventures of Robin Hood, The (1938)",5.0
325,325,Alice in Wonderland (1951),5.0
...,...,...,...
9537,9537,"Wizard of Oz, The (1939)",5.0
9545,9545,"Wolf Man, The (1941)",5.0
9615,9615,X-Men (2000),5.0
9656,9656,Young Frankenstein (1974),5.0


In [73]:
NON_MOVIE = 0
HIGH_SCORE = 4

def get_score_movie(data_frame, userid, score = -1):
  #userId에 따른 영화 추천
  user_movie = data_frame.loc[userid]

  #series를 DataFrame으로 변환
  df_movies_score = pd.DataFrame(data={'title':user_movie.index, 'score':user_movie.values})
  df_movies_score = df_movies_score.reset_index()

  #만약 안 본 영화이면,
  if score == 0:
    return df_movies_score[df_movies_score['score']==0]
  
  # score인자 이상의 점수를 받은영화이면.
  return df_movies_score[df_movies_score['score']>=score].sort_values('score', ascending=False)
  

get_score_movie(df_ratings_movies_pivot, TARGET_USER, HIGH_SCORE)

,index,title,score
3564,3564,Goodfellas (1990),5.0
8240,8240,"Sword in the Stone, The (1963)",5.0
5157,5157,"Lock, Stock & Two Smoking Barrels (1998)",5.0
5149,5149,Live and Let Die (1973),5.0
8001,8001,Star Wars: Episode IV - A New Hope (1977),5.0
...,...,...,...
4017,4017,Hook (1991),4.0
4010,4010,"Honey, I Shrunk the Kids (1989)",4.0
3845,3845,Heat (1995),4.0
3661,3661,Grumpier Old Men (1995),4.0


###3.2.4 2, 3번을 이용한 추천 영화 목록 만들기

In [74]:
def get_min_max(df_non_movies, df_highscore_movies):
  # 두 DataFrame을 비요하여 짧은 길이의 dataframe을 앞에서, 긴 dataframe을 뒤에서 반환
  if len(df_non_movies) < len(df_highscore_movies):
    return df_non_movies, df_highscore_movies
  else:
    return df_highscore_movies, df_non_movies

def get_recomman_movies(df_non_movies, df_highscore_movies):
  #반복문을 최소화 하기 위해 적은 영화를 가지고 있는 dataframe얻기
  df_min_movie, df_max_movie = get_min_max(df_non_movies, df_highscore_movies)

  df_recommand_movies = pd.DataFrame()
  for movie_name in df_min_movie['title']:
    df_movie_title = df_max_movie[df_max_movie['title']==movie_name]
    if not df_movie_title.empty:
      df_recommand_movies = df_recommand_movies.append(df_movie_title['title'].to_frame())
  return df_recommand_movies

USER1 = 1
USER2 = 266
df_non_movie = get_score_movie(df_ratings_movies_pivot, USER1, NON_MOVIE)
df_highscore_movies = get_score_movie(df_ratings_movies_pivot, USER2, HIGH_SCORE)
get_recomman_movies(df_non_movie, df_highscore_movies)

,title
74,2001: A Space Odyssey (1968)
3065,"Fisher King, The (1991)"
3226,Friday (1995)
8157,Summer of Sam (1999)
8116,Stripes (1981)
3769,Hard-Boiled (Lat sau san taam) (1992)
3847,Heathers (1989)
4134,"Hunt for Red October, The (1990)"
7241,Ronin (1998)
7220,Roger & Me (1989)


In [75]:
# 추천받을 영화 수 
RECOMMAND_MOVIE = 10

#추천 받을 사용자
TARGET_USER = 1

#안본 영화
NON_MOVIE = 0

#추천 받을 최 저점
HIGH_SCORE = 5

#1. 유사도가 높은 사용자 추출
user_sim = get_user_similarity(df_ratings_movies_pivot, TARGET_USER)

#2. 추천 받을 사람의 안본 영화 추출
df_non_movies = get_score_movie(df_ratings_movies_pivot, TARGET_USER, NON_MOVIE)

#추천 받을 영화 저장소를 일단 만들어 놓음
df_recommand_movies = pd.DataFrame()

for userId in user_sim.index:
  #3. 추천하는 사람의 4점 이상 영화 추출
  df_highscore_movies = get_score_movie(df_ratings_movies_pivot, userId, HIGH_SCORE)
  #추천 영화 목록 만들기
  df_recommand_movie = get_recomman_movies(df_non_movie, df_highscore_movies)
  df_recommand_movies = df_recommand_movies.append(df_recommand_movie)

#6. 빈도수 구하기
df_class_movies = df_recommand_movies.groupby('title')
#7. 빈도수가 높은 영화 추천
df_class_movies['title'].count().sort_values(ascending=False)[:RECOMMAND_MOVIE]

title
Godfather, The (1972)                                                          7
Aliens (1986)                                                                  6
Jaws (1975)                                                                    6
Blade Runner (1982)                                                            6
Sixth Sense, The (1999)                                                        5
Godfather: Part II, The (1974)                                                 4
Contact (1997)                                                                 4
Fisher King, The (1991)                                                        4
Fish Called Wanda, A (1988)                                                    4
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)    4
Name: title, dtype: int64

###3.2.5 가중치
> 유사도와 평점을 곱하여 새로운 점수를 만들고 이를 기준으로 추천

In [76]:
def get_recommand_movies(df_non_movies, df_highscore_movies, user_similarity):
  #반복문을 최소화 하기 위해 적은 영화를 가지고 있는 dataframe얻기
  df_min_movie, df_max_movie = get_min_max(df_non_movies, df_highscore_movies)

  df_recommand_movies = pd.DataFrame(columns = ['title', 'score'])

  #for movie_name in df_min_movie['title']:
   #zip함수에 인자로 전달된 두 값을 반환
  for title, score in zip(df_min_movie['title'], df_min_movie['score']): 
    df_movie_title = df_max_movie[df_max_movie['title']==title]

    if not df_movie_title.empty:
      #score가 df_non_movies인 경우 0을 갖게 된다.
      if score == NON_MOVIE:
        #따라서 df_highscore_movies의 score정보로 변경된다.
        score = df_max_movie['score']
      df_movie_title['score']= score * user_similarity
      df_recommand_movies = df_recommand_movies.append(df_movie_title)

  return df_recommand_movies

USER1 = 1
USER2 = 266

df_non_movies = get_score_movie(df_ratings_movies_pivot, USER1, NON_MOVIE)
df_highscore_movies = get_score_movie(df_ratings_movies_pivot, USER2, HIGH_SCORE)

get_recommand_movies(df_non_movies, df_highscore_movies, user_sim.values[0])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,title,score,index
74,2001: A Space Odyssey (1968),1.787039,74.0
333,Aliens (1986),1.787039,333.0
6829,"Producers, The (1968)",1.787039,6829.0
6955,Raising Arizona (1987),1.787039,6955.0
6991,Real Genius (1985),1.787039,6991.0
7043,"Ref, The (1994)",1.787039,7043.0
7220,Roger & Me (1989),1.787039,7220.0
7241,Ronin (1998),1.787039,7241.0
8116,Stripes (1981),1.787039,8116.0
8157,Summer of Sam (1999),1.787039,8157.0


In [77]:
RECOMMAND_MOVIE = 10
TARGET_USER = 1
NON_MOVIE = 0
HIGH_SCORE = 4

user_sim = get_user_similarity(df_ratings_movies_pivot, TARGET_USER)
df_non_movies = get_score_movie(df_ratings_movies_pivot, TARGET_USER, NON_MOVIE)

df_recommand_movies = pd.DataFrame()

# for userId in user_sim.index:
for userId, usr_sim in zip(user_sim.index, user_sim.values):
  df_highscore_movies = get_score_movie(df_ratings_movies_pivot, userId, HIGH_SCORE)
  df_recommand_movie = get_recommand_movies(df_non_movies, df_highscore_movies, usr_sim)
  df_recommand_movies = df_recommand_movies.append(df_recommand_movie)

df_class_movies = df_recommand_movies.groupby('title')
print( df_class_movies['score'].count().sort_values(ascending=False)[:RECOMMAND_MOVIE] )
print( df_class_movies['score'].sum().sort_values(ascending=False)[:RECOMMAND_MOVIE] )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

title
Terminator 2: Judgment Day (1991)            9
Aliens (1986)                                8
Godfather, The (1972)                        8
Godfather: Part II, The (1974)               8
Sixth Sense, The (1999)                      8
Die Hard (1988)                              8
Blade Runner (1982)                          7
Gattaca (1997)                               7
Hunt for Red October, The (1990)             7
Twelve Monkeys (a.k.a. 12 Monkeys) (1995)    7
Name: score, dtype: int64
title
Terminator 2: Judgment Day (1991)            13.146918
Godfather, The (1972)                        13.134550
Aliens (1986)                                12.936063
Sixth Sense, The (1999)                      12.432916
Godfather: Part II, The (1974)               12.104995
Blade Runner (1982)                          11.556014
Die Hard (1988)                              10.889409
Gattaca (1997)                               10.561737
Hunt for Red October, The (1990)             10.3420

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

df_ratings_movies_pivot_T = df_ratings_movies_pivot.transpose()

movies_sim = cosine_similarity(df_ratings_movies_pivot_T, df_ratings_movies_pivot_T)

df_movies_sim = pd.DataFrame(data=movies_sim, index=df_ratings_movies_pivot.columns, columns=df_ratings_movies_pivot.columns)
df_movies_sim

##3.4 개인별 영화 추천

In [79]:
import numpy as np
from sklearn.metrics import mean_squared_error

def get_mse(pred, data_frame):
  rating_pred_nonzero = pred[data_frame.nonzero()]
  df_rating_nonzero = data_frame[data_frame.nonzero()]

  return mean_squared_error(rating_pred_nonzero, df_rating_nonzero)

def predict_rating_top(ratings_arr, item_sim, top=10):
  pred = np.zeros(ratings_arr.shape)

  for row in range(ratings_arr.shape[0]):
    top_n_item = np.argsort(ratings_arr[row, :])[::-1][:top]
    for col in range(ratings_arr.shape[1]):
      pred[row, col] = item_sim[:, col][top_n_item].dot(ratings_arr[row, :][top_n_item])
      pred[row, col] /= np.sum(np.abs(item_sim[col, :][top_n_item]))
  
  return pred

ratings_pred = predict_rating_top(df_ratings_movies_pivot.values, movies_sim)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars


In [80]:
print(get_mse(ratings_pred, df_ratings_movies_pivot.values))

3.0918230908381696


In [81]:
df_ratings_pred = pd.DataFrame(data=ratings_pred, index=df_ratings_movies_pivot.index, columns=df_ratings_movies_pivot.columns)
df_ratings_pred

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...All the Marbles (1981),...And Justice for All (1979),00 Schneider - Jagd auf Nihil Baxter (1994),1-900 (06) (1994),10 (1979),10 Cent Pistol (2015),10 Cloverfield Lane (2016),10 Items or Less (2006),10 Things I Hate About You (1999),10 Years (2011),"10,000 BC (2008)",100 Girls (2000),100 Streets (2016),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),101 Dalmatians II: Patch's London Adventure (2003),101 Reykjavik (101 Reykjavík) (2000),102 Dalmatians (2000),10th & Wolf (2006),"10th Kingdom, The (2000)","10th Victim, The (La decima vittima) (1965)","11'09""01 - September 11 (2002)",11:14 (2003),"11th Hour, The (2007)",12 Angry Men (1957),12 Angry Men (1997),12 Chairs (1971),12 Chairs (1976),12 Rounds (2009),12 Years a Slave (2013),...,Zathura (2005),Zatoichi and the Chest of Gold (Zatôichi senryô-kubi) (Zatôichi 6) (1964),Zazie dans le métro (1960),Zebraman (2004),"Zed & Two Noughts, A (1985)",Zeitgeist: Addendum (2008),Zeitgeist: Moving Forward (2011),Zeitgeist: The Movie (2007),Zelary (2003),Zelig (1983),Zero Dark Thirty (2012),Zero Effect (1998),"Zero Theorem, The (2013)",Zero de conduite (Zero for Conduct) (Zéro de conduite: Jeunes diables au collège) (1933),Zeus and Roxanne (1997),Zipper (2015),Zodiac (2007),Zombeavers (2014),Zombie (a.k.a. Zombie 2: The Dead Are Among Us) (Zombi 2) (1979),Zombie Strippers! (2008),Zombieland (2009),Zone 39 (1997),"Zone, The (La Zona) (2007)",Zookeeper (2011),Zoolander (2001),Zoolander 2 (2016),Zoom (2006),Zoom (2015),Zootopia (2016),Zulu (1964),Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.000000,5.0,5.0,NaN,5.0,NaN,5.000000,5.0,5.000000,5.0000,NaN,5.0,5.000000,NaN,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.0,5.0,5.000000,NaN,5.000000,NaN,5.0,5.000000,NaN,5.000000,5.0,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,NaN,5.000000,5.0,5.000000,5.000000,5.00000,5.0,5.000000,5.000000,5.000000,5.00000,5.0,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.0,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.0000,NaN,5.00000,5.00000,5.00000,5.000000,5.0
2,4.720443,4.5,4.5,NaN,NaN,4.812939,4.690703,4.5,4.770632,4.7328,NaN,4.5,4.678439,NaN,4.719721,4.747244,4.768155,4.681967,4.725818,4.740276,4.777483,4.793534,4.740276,4.728806,4.688742,4.5,4.5,4.644355,NaN,4.722047,NaN,4.5,4.638728,NaN,4.698629,4.5,4.852789,4.852789,4.724942,4.788318,...,4.701518,4.852789,4.857747,4.679575,4.5,4.883227,4.883227,4.85827,4.5,4.626241,4.752508,4.748769,4.78658,4.5,4.732724,4.740276,4.743723,4.726942,4.606996,4.876446,4.744094,4.732724,4.883095,4.747174,4.743637,4.753263,NaN,4.740276,4.788298,4.680554,4.740276,4.631718,4.736199,4.7239,4.818802,4.75551,4.74743,4.69594,4.736724,NaN
3,5.000000,5.0,5.0,NaN,NaN,NaN,5.000000,NaN,5.000000,5.0000,5.0,5.0,5.000000,5.0,NaN,NaN,5.000000,5.000000,5.000000,NaN,5.000000,5.000000,NaN,5.000000,5.000000,NaN,NaN,5.000000,NaN,5.000000,5.0,5.0,5.000000,NaN,5.000000,NaN,NaN,NaN,5.000000,5.000000,...,5.000000,NaN,NaN,5.000000,5.0,5.000000,5.000000,5.00000,NaN,5.000000,5.000000,5.000000,5.00000,5.0,5.000000,NaN,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,NaN,5.000000,5.000000,NaN,NaN,5.000000,5.000000,NaN,5.000000,5.000000,5.0000,NaN,5.00000,5.00000,5.00000,5.000000,5.0
4,5.000000,5.0,5.0,NaN,5.0,NaN,5.000000,5.0,5.000000,5.0000,NaN,5.0,5.000000,NaN,5.000000,5.000000,5.000000,5

In [82]:
def get_unseen_movies(data_frame, userId):
  # userid에 설정된 모든 영화 추출
  user_rating = data_frame.loc[userId]
  # 안본 영화 추출
  unseen_list = user_rating[user_rating==0].index
  # 리스트화
  return unseen_list.tolist()

unseen_list = get_unseen_movies(df_ratings_movies_pivot, 1)

In [83]:
def recomm_by_user(pred_df, userId, unseen_list, top_n=10):
  # 사용자의 예측 점수가 포함된 영화
  movies = pred_df.loc[userId]
  # 안본 영화 목록
  unseen_movies = movies[unseen_list]
  # 상위 top n
  recomm_movies = unseen_movies.sort_values(ascending=False)[:top_n]
  return recomm_movies

recomm_by_user(df_ratings_pred, 1, unseen_list, 10)

title
Titanic (1997)                                             5.0
Food, Inc. (2008)                                          5.0
Darkest Hour (2017)                                        5.0
Guys and Dolls (1955)                                      5.0
Smilla's Sense of Snow (1997)                              5.0
Erik the Viking (1989)                                     5.0
Bug's Life, A (1998)                                       5.0
National Security (2003)                                   5.0
National Lampoon's Van Wilder (2002)                       5.0
Wallace & Gromit in The Curse of the Were-Rabbit (2005)    5.0
Name: 1, dtype: float64